In [1]:
#!/usr/bin/env python3
"""
pipeline_ev20_alert_no_tls.py

- SUMO tự động gán 20% xe điện (vType="car_eidm") dựa trên <device.battery.probability> trong eidm-1.xml.
- Loại bỏ hoàn toàn việc patch 20% bằng Python.
- Mỗi bước:
    1) Remove mọi xe vừa bị teleport.
    2) “Dọn” jam: nếu bất kỳ edge nào có xe đứng chờ > 0, reroute các xe đó ngay lập tức.
    3) Check SOC của các EV; khi SOC ≤ 20% maxCharge → log lần đầu thời gian + vị trí.
- Thêm các output SUMO:
    --battery-output (precision=4),
    --edge-data-output (period=60),
    --lane-data-output (period=60).
- Kết quả log EV chạm ngưỡng báo động nằm trong “battery_alert_{tag}.txt”.
"""

from __future__ import annotations

import os
import sys
import time
import re
from pathlib import Path
from typing import Optional

# Import TraCI
import traci

# ---------------------------------------------------------------------------
#  Cấu hình chung
# ---------------------------------------------------------------------------

# Tối ưu số threads cho SUMO
THREADS = min(1, os.cpu_count() or 1)
os.environ["OMP_NUM_THREADS"] = str(THREADS)

# Nếu cần, tùy chỉnh LD_LIBRARY_PATH / SUMO_HOME
os.environ["LD_LIBRARY_PATH"] = os.path.expanduser("~/Libs/libnsl")
os.environ["SUMO_HOME"]     = os.path.expanduser("~/Envs/sumo-env/lib/python3.10/site-packages/sumo")

# Đường dẫn tới mạng SUMO và vTypes EV
NET_XML    = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
VTYPES_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-combine-ecar.xml"

# Thư mục lưu kết quả mô phỏng
SIM_DIR     = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/03-06-ev20-segmented")
TRIPS_DIR   = SIM_DIR / "trips"
OUTPUTS_DIR = SIM_DIR / "outputs"
STATES_DIR  = SIM_DIR / "states"
LOGS_DIR    = SIM_DIR / "logs"

for p in (TRIPS_DIR, OUTPUTS_DIR, STATES_DIR, LOGS_DIR):
    p.mkdir(parents=True, exist_ok=True)

# Tham số khác
HOUR             = 3600
STEP_LENGTH      = 1.0
TIME_TO_TELEPORT = "120"
TRACI_PORT       = 8813  # Port dùng cho TraCI

# Thư mục chứa các file route-segment gốc (chưa need patch EV)
ROUTES_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/smooth-routes")

# Tham số EV: lấy từ vType car_eidm trong eidm-1.xml
EV_TYPE  = "ecar_eidm"

# Ngưỡng báo động: 20% dung lượng
BATTERY_ALERT_RATIO = 0.2

# ---------------------------------------------------------------------------
#  Hàm kiểm tra outputs đã hoàn thành cho mỗi segment
# ---------------------------------------------------------------------------

def outputs_complete(tag: str) -> bool:
    """
    Kiểm tra nếu thư mục outputs/segment_{tag} đã có đủ 6 file:
    summary.xml, tripinfo.xml, statistics.xml, vehRoutes.xml, laneChanges.xml, collisions.xml
    và trong statistics.xml có thẻ <performance> (SUMO đã chạy xong).
    """
    outdir = OUTPUTS_DIR / f"segment_{tag}"
    files = [
        outdir / "summary.xml",
        outdir / "tripinfo.xml",
        outdir / "statistics.xml",
        outdir / "vehRoutes.xml",
        outdir / "laneChanges.xml",
        outdir / "collisions.xml",
    ]
    if not all(f.exists() and f.stat().st_size > 0 for f in files):
        return False

    try:
        content = (outdir / "statistics.xml").read_text(encoding="utf-8")
    except Exception:
        return False

    remainder = content.split("-->", maxsplit=1)[-1]
    return "<performance" in remainder

def state_path(end_time: int) -> Path:
    """
    Đường dẫn tới file state_{seconds}.xml.gz.
    Ví dụ nếu kết thúc tại 07:20:00 (7*3600 + 20*60 = 26400 giây),
    file sẽ là "state_00026400.xml.gz".
    """
    return STATES_DIR / f"state_{end_time:08d}.xml.gz"

# ---------------------------------------------------------------------------
#  Bước 1: Scan tất cả route-segment (.xml) và xây dựng danh sách segments
# ---------------------------------------------------------------------------

# Pattern filename: routes_HH_XofY.xml (ví dụ: routes_07_2of3.xml)
segment_pattern = re.compile(r"routes_(\d{2})_(\d+)of(\d+)\.xml$")
segments = []

for route_file in ROUTES_DIR.glob("*.xml"):
    m = segment_pattern.match(route_file.name)
    if not m:
        continue
    hour  = int(m.group(1))
    part  = int(m.group(2))
    total = int(m.group(3))
    seg_duration = HOUR // total                # ví dụ 3600//3 = 1200
    begin  = hour * HOUR + (part - 1) * seg_duration
    end    = hour * HOUR + part * seg_duration
    tag    = f"{hour:02d}_{part}of{total}"
    segments.append({
        "original_path": route_file,  # đường dẫn file gốc
        "hour": hour,
        "begin": begin,
        "end": end,
        "tag": tag
    })

if not segments:
    sys.exit("[ERROR] Không tìm thấy route segments trong thư mục")

# Sắp xếp segments theo thời gian bắt đầu (begin)
segments.sort(key=lambda seg: seg["begin"])

# ---------------------------------------------------------------------------
#  Bước 2: Xác định điểm resume (nếu đã có kết quả cũ)
# ---------------------------------------------------------------------------

prev_state: Optional[Path] = None
start_idx = 0
for idx in reversed(range(len(segments))):
    seg = segments[idx]
    if outputs_complete(seg["tag"]):
        start_idx  = idx + 1
        prev_state = state_path(seg["end"])
        break

print(f"Resume from segment index {start_idx} (prev_state={prev_state})")

# ---------------------------------------------------------------------------
#  Bước 3: Main loop – cho từng segment: chạy SUMO + TraCI (không patch EV)
# ---------------------------------------------------------------------------

for idx in range(start_idx, len(segments)):
    seg   = segments[idx]
    tag   = seg["tag"]    # ví dụ "07_2of3"
    begin = seg["begin"]  # ví dụ 25200 + 1200 = 26400
    end   = seg["end"]    # ví dụ 27600

    outdir = OUTPUTS_DIR / f"segment_{tag}"
    outdir.mkdir(exist_ok=True)

    # Nếu đã hoàn thành output → skip
    if outputs_complete(tag):
        print(f"[Seg {tag}] outputs already complete – skip")
        prev_state = state_path(end)
        continue

    # --- Bước 3.1: Không patch, dùng nguyên route gốc ---
    patched_route = seg["original_path"]
    print(f"[Seg {tag}] Dùng nguyên route: {patched_route.name}")

    # --- Bước 3.2: Chuẩn bị các file output SUMO ---
    summary_xml     = outdir / "summary.xml"
    tripinfo_xml    = outdir / "tripinfo.xml"
    statistics_xml  = outdir / "statistics.xml"
    vehroutes_xml   = outdir / "vehRoutes.xml"
    lanechanges_xml = outdir / "laneChanges.xml"
    collision_xml   = outdir / "collisions.xml"

    battery_xml  = outdir / "battery.xml"
    edgedata_xml = outdir / "edgedata.xml"
    lanedata_xml = outdir / "lanedata.xml"

    state_file   = state_path(end)
    virtual_end  = end + 1  

    sumo_log = LOGS_DIR / f"sumo_{tag}.log"
    sumo_cmd = [
        "sumo",  
        "-n", NET_XML,
        "-r", str(patched_route),
        "--threads", str(THREADS),
        "--step-length", str(STEP_LENGTH),
        "--begin", str(begin),
        "--end", str(virtual_end),

        "--additional-files", str(VTYPES_XML),

        "--summary-output",   str(summary_xml),
        "--tripinfo-output",  str(tripinfo_xml),
        "--statistic-output", str(statistics_xml),
        "--vehroute-output",  str(vehroutes_xml),
        "--lanechange-output",str(lanechanges_xml),
        "--collision-output", str(collision_xml),
        "--time-to-teleport", TIME_TO_TELEPORT,
        "--ignore-junction-blocker", "20",
        "--lateral-resolution", "0.4",
        "--ignore-route-errors",
        "--no-step-log", "--duration-log.statistics",
        "--save-state.times", str(end),
        "--save-state.files", str(state_file),
        "--xml-validation", "never",
        "--log", str(sumo_log),

        # Thêm output mới:
        "--battery-output", str(battery_xml),
        "--battery-output.precision", "4",
        "--edgedata-output", str(edgedata_xml),
        "--lanedata-output", str(lanedata_xml),
    ]

    # Nếu có state từ segment trước, load vào
    if prev_state and prev_state.exists():
        sumo_cmd += ["--load-state", str(prev_state)]

    # Ghi log bắt đầu SUMO
    with open(sumo_log, "a", encoding="utf-8") as flog:
        flog.write(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] [Seg {tag}] SUMO start {begin} → {end}\n")
    print(f"[Seg {tag}] SUMO start {begin} → {end}")

    # --- Bắt đầu SUMO qua TraCI ---
    traci.start(sumo_cmd, port=TRACI_PORT)

    # --- Bước 3.4: Lấy danh sách EV thực (có battery) và xác định ngưỡng alert ---
    all_veh_ids = traci.vehicle.getIDList()
    ev_ids = []
    ev_max_charge = {}
    for vid in all_veh_ids:
        try:
            cap = float(traci.vehicle.getParameter(vid, "device.battery.capacity"))
            if cap > 0.0:
                ev_ids.append(vid)
                ev_max_charge[vid] = cap
        except traci.exceptions.TraCIException:
            # Nếu không có tham số battery.capacity → không phải EV
            continue

    ev_alert_threshold = {vid: ev_max_charge[vid] * BATTERY_ALERT_RATIO for vid in ev_ids}
    ev_alert_logged   = {vid: False for vid in ev_ids}

    battery_alert_log = LOGS_DIR / f"battery_alert_{tag}.txt"
    with open(battery_alert_log, "w", encoding="utf-8") as flog:
        flog.write("vehID,time(s),x,y,roadID,charge,maxCharge\n")

    sumo_start_time = time.time()

    # -----------------------------------------------------------------------
    #  Bước 3.5: Vòng simulation mỗi step
    # -----------------------------------------------------------------------
    for step in range(begin, end + 1):
        traci.simulationStep()

        # 1) Remove ngay các xe vừa teleport
        try:
            teleported = traci.simulation.getEndingTeleportIDList()
            for vid in teleported:
                traci.vehicle.remove(vid)
        except Exception:
            pass

        # 2) “Dọn” jam: nếu edge có halting > 0 → reroute tất cả xe trên edge đó
        try:
            for eid in traci.edge.getIDList():
                if traci.edge.getLastStepHaltingNumber(eid) > 0:
                    for vid in traci.edge.getLastStepVehicleIDs(eid):
                        try:
                            curr_edge = traci.vehicle.getRoadID(vid)
                            old_route  = traci.vehicle.getRoute(vid)
                            if not old_route:
                                continue
                            dest_edge  = old_route[-1]
                            route_info = traci.simulation.findRoute(curr_edge, dest_edge)
                            new_edges  = route_info.edges
                            if new_edges:
                                traci.vehicle.setRoute(vid, new_edges)
                        except Exception:
                            pass
        except Exception:
            pass

        # 3) Check SOC của mỗi EV; nếu ≤ ngưỡng báo động → log lần đầu
        for vid in ev_ids:
            if ev_alert_logged[vid]:
                continue
            try:
                curr_charge = float(traci.vehicle.getParameter(vid, "device.battery.chargeLevel"))
            except traci.exceptions.TraCIException:
                # Nếu không thể đọc chargeLevel → bỏ qua lần sau
                ev_alert_logged[vid] = True
                continue

            if curr_charge <= ev_alert_threshold[vid]:
                x, y  = traci.vehicle.getPosition(vid)
                road  = traci.vehicle.getRoadID(vid)
                max_c = ev_max_charge[vid]
                with open(battery_alert_log, "a", encoding="utf-8") as flog:
                    flog.write(f"{vid},{step},{x:.2f},{y:.2f},{road},{curr_charge:.4f},{max_c:.4f}\n")
                ev_alert_logged[vid] = True

    # Kết thúc TraCI → SUMO sẽ tự lưu state
    traci.close()
    sumo_end_time = time.time()
    sumo_runtime = sumo_end_time - sumo_start_time
    with open(sumo_log, "a", encoding="utf-8") as flog:
        flog.write(f"(-----FINISHED----- SUMO in {sumo_runtime:.2f}s)\n")

    print(f"[Seg {tag}] finished (state saved → {state_path(end)})\n")
    prev_state = state_path(end)

print("All segments simulations finished")


Resume from segment index 0 (prev_state=None)
[Seg 00_1of1] Dùng nguyên route: routes_00_1of1.xml
[Seg 00_1of1] SUMO start 0 → 3600
 Retrying in 1 seconds
***Starting server on port 8813 ***
Loading net-file from '/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml' ...Interrupt signal received, trying to exit gracefully.


Traceback (most recent call last):
  File "/home/hoai-linh.dao/Envs/sumo-env/bin/sumo", line 8, in <module>
    sys.exit(sumo())
  File "/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/site-packages/sumo/__init__.py", line 28, in <lambda>
    return lambda: sys.exit(subprocess.call([os.path.join(SUMO_HOME, 'bin', app)] + sys.argv[1:], env=os.environ))
  File "/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/subprocess.py", line 347, in call
    return p.wait(timeout=timeout)
  File "/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/subprocess.py", line 1209, in wait
    return self._wait(timeout=timeout)
  File "/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/home/hoai-linh.dao/Envs/sumo-env/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt


KeyboardInterrupt: 